In [1]:
import numpy as np
import pandas as pd
from statsmodels.stats.multitest import fdrcorrection

In [2]:
gencode_23 = pd.read_csv('../data/processed/ensembl_main.csv', usecols=[0, 1], index_col=0).squeeze('columns')

In [3]:
gencode_23 = gencode_23.replace('pk', 'MAPK20', regex=False)

In [4]:
def sep_normal(df):
    def samplify(df):
        samples_n = df.columns[df.columns.str.endswith('11')].to_list()
        samples_c = df.columns[df.columns.str.endswith('01')].to_list()

        return samples_n, samples_c
    
    samples_n, samples_c = samplify(df)
        
    return df[samples_n], df[samples_c]

In [5]:
tcga_groups = (
    pd.read_excel('../data/processed/tcga_groups.xlsx', index_col=0)
    .query('normal > 20')
    .index.to_list()
)

In [6]:
ress = []

for ctype in tcga_groups:
    tpm = pd.read_csv(f'../data/processed/tcga_type_data/{ctype}_tpm.csv', index_col=[0, 1])
    psi = np.abs(np.around((tpm / tpm.groupby(level=0).transform('sum')).fillna(0), 3))

    tpm = tpm.droplevel(0)
    psi = psi.droplevel(0)

    saturn = (
        pd.read_csv(f'../results/saturn/{ctype}_saturn.csv', index_col=0, usecols=[0, 5, 6, 7, 8])
        .rename({'pval': 'dtu_pval',
                 'regular_FDR': 'dtu_fdr_local',
                 'empirical_pval': 'dtu_epval',
                 'empirical_FDR': 'dtu_efdr_local'}, axis=1)
    )
    edger = (
        pd.read_csv(f'../results/general/{ctype}_edger.csv', index_col=0, usecols=[0, 1, 4, 5])
        .rename({'logFC': 'log2fc',
                 'PValue': 'dte_pval',
                 'FDR': 'dte_fdr_local'}, axis=1)
    )
    tpm = tpm[tpm.index.isin(saturn.index)]
    psi = psi[psi.index.isin(saturn.index)]
    edger = edger[edger.index.isin(saturn.index)]

    tpm_n, tpm_c = sep_normal(tpm)
    psi_n, psi_c = sep_normal(psi)

    res = (
        pd.DataFrame(index=tpm.index)
        .assign(ctype=ctype,
                gene_name=tpm.index.map(gencode_23),
                ie_n=tpm_n.mean(1),
                ie_c=tpm_c.mean(1),
                die=tpm_c.mean(1) - tpm_n.mean(1),
                if_n=psi_n.mean(1),
                if_c=psi_c.mean(1),
                dif=psi_c.mean(1) - psi_n.mean(1))
        .join(saturn)
        .join(edger)
    )
    res['log2fc'] *= -1
    
    res = res.reset_index().dropna(subset='dtu_pval')

    ress.append(res)
    
ress = pd.concat(ress)

In [7]:
ress['dtu_efdr_global'] = fdrcorrection(ress['dtu_epval'])[1]
ress['dte_fdr_global'] = fdrcorrection(ress['dte_pval'])[1]

ress['is_dtu'] = (abs(ress['dif']) > 0.1) & (ress['dtu_efdr_global'] < 0.05)
ress['is_dte'] = ress['dte_fdr_global'] < 0.05

In [8]:
ress = ress[['ctype', 'gene_name', 'ensembl_trs_id', 'ie_n', 'ie_c', 'die', 'log2fc', 
             'if_n', 'if_c', 'dif', 'dtu_pval', 'dtu_fdr_local', 'dtu_epval', 'dtu_efdr_local', 
             'dtu_efdr_global', 'is_dtu', 'dte_pval', 'dte_fdr_local', 'dte_fdr_global', 'is_dte']]

ress.iloc[:, [3, 4, 5, 6, 7, 8, 9]] = np.around(ress.iloc[:, [3, 4, 5, 6, 7, 8, 9]], 3)

In [9]:
ress_dict = {
    ctype: res
    for ctype, res in ress.groupby('ctype')
}
ress_dtu = []

for ctype in tcga_groups:
    res = ress_dict[ctype]
    
    res.to_csv(f'../results/general/{ctype}_quant.csv', index=False)
    
    res_dtu = res.loc[res['is_dtu'], ['gene_name', 'ensembl_trs_id', 'dif', 'log2fc', 'is_dte']].set_index('gene_name')
    
    res_dtu_up = res_dtu[res_dtu['dif'] > 0.1]
    res_dtu_down = res_dtu[res_dtu['dif'] < 0.1]
    
    # By joining on gene name, create all possible combinations of up and downregulated transcripts = isoform switches
    res_dtu = (
        res_dtu_up.join(res_dtu_down, how='inner', rsuffix='_n')
        .sort_index()
        .rename({'ensembl_trs_id': 'ensembl_trs_id_c', 
                 'dif': 'dif_c',
                 'log2fc': 'log2fc_c', 
                 'is_dte': 'is_dte_c'}, axis=1)
    )
    res_dtu['ctype'] = ctype
    res_dtu = res_dtu.reset_index()

    ress_dtu.append(res_dtu)
    
ress_dtu = pd.concat(ress_dtu)

ress_dtu = ress_dtu[['ctype'] + ress_dtu.columns[:-1].to_list()]

In [10]:
ress_dtu_dict = {
    ctype: res_dtu
    for ctype, res_dtu in ress_dtu.groupby('ctype')
}
ress_freq = []

for ctype in tcga_groups:
    res = ress_dtu_dict[ctype]
    
    tpm = pd.read_csv(f'../data/processed/tcga_type_data/{ctype}_tpm.csv', index_col=1).drop('gene_name', axis=1)
    tpm_n, tpm_c = sep_normal(tpm)
    
    res['freq_n'] = np.count_nonzero(tpm_n.loc[res['ensembl_trs_id_n']] > tpm_n.loc[res['ensembl_trs_id_c']].values, axis=1) / len(tpm_n.columns)
    res['freq_c'] = np.count_nonzero(tpm_c.loc[res['ensembl_trs_id_c']] > tpm_c.loc[res['ensembl_trs_id_n']].values, axis=1) / len(tpm_c.columns)
    
    ress_freq.append(res)
    
ress_freq = pd.concat(ress_freq)

In [11]:
ress_freq['freq_prod'] = ress_freq['freq_n'] * ress_freq['freq_c']
ress_freq.iloc[:, -3:] = np.around(ress_freq.iloc[:, -3:], 3)

In [12]:
ress_freq.to_csv('../results/switches.csv', index=False)